In [ ]:
# definición del set de datos y los nombres de sus atributos (ejemplo visto en clases)
headers = ['Outlook', 'Temperature', 'Humidity', 'Wind', 'Plays']
data=[['Sunny','Hot','High','Weak','No'],
      ['Sunny','Hot','High','Strong','No'],
      ['Overcast','Hot','High','Weak','Yes'],
      ['Rain','Mild','High','Weak','Yes'],
      ['Rain','Cool','Normal','Weak','Yes'],
      ['Rain','Cool','Normal','Strong','No'],
      ['Overcast','Cool','Normal','Strong','Yes'],
      ['Sunny','Mild','High','Weak','No'],
      ['Sunny','Cool','Normal','Weak','Yes'],
      ['Rain','Mild','Normal','Weak','Yes'],
      ['Sunny','Mild','Normal','Strong','Yes'],
      ['Overcast','Mild','High','Strong','Yes'],
      ['Overcast','Hot','Normal','Weak','Yes'],
      ['Rain','Mild','High','Strong','No']]

In [ ]:
# nodo de un árbol, contiene el test que realiza (col), y el valor del atributo que llevó a este nodo
class decision_node:
    def __init__(self,col=-1, value = '', branches=None, results=None):
        self.col=col
        self.value = value
        self.results=results
        self.branches = branches

In [ ]:
# división del set de datos de acuerdo a los valores de un atributo
def divide_set(rows,column):
    sets = dict()
    for row in rows:
        if row[column] not in sets.keys():
            sets[row[column]] = []
        sets[row[column]].append(row)
    return sets
divide_set(data,0)

In [ ]:
# conteo de registros pertenecientes a cada clase
def unique_counts(rows):
    results={}
    for row in rows:
        r = row[-1]
        if r not in results: 
            results[r]=0
        results[r] += 1
    return results
unique_counts(data)

In [ ]:
def entropy(rows):
    from math import log2
    results=unique_counts(rows)
    # Now calculate the entropy
    ent=0.0
    for r in results.keys():
        p=float(results[r])/len(rows) 
        ent=ent-p*log2(p)
    return ent
entropy(data)

In [ ]:
# construcción del árbol usando ganancia de información 
def build_tree(rows, headers, value=''):
    if len(rows) == 0: 
        return decision_node(value = value)
    elif len(set([row[-1] for row in rows])) == 1:
        return decision_node(value = value, results=unique_counts(rows))
    current_score = entropy(rows)
    best_gain = 0.0
    best_criteria = None
    best_sets = None
    column_count = len(rows[0]) - 1
    for col in range(0, column_count):
        sets = divide_set(rows, col)
        gain = current_score
        for key in sets.keys():
            p = float(len(sets[key])) / len(rows)
            gain -= p*entropy(sets[key])
        if gain > best_gain:
            best_gain = gain
            best_criteria = headers[col]
            best_sets = sets
    branches = []
    for s in best_sets:
        branches.append(build_tree(best_sets[s], headers, s))
    return decision_node(col=best_criteria, value = value, branches=branches)

In [ ]:
def print_tree(tree, indent = ''):
    if tree.results != None:
        print(indent + str(tree.results))
    else:
        if tree.value == '':
            print(indent + tree.col + "?")
        for branch in tree.branches:
            to_print = indent + '  ' + branch.value + " -> "
            if branch.col != -1:
                print(to_print + branch.col + "?")
                print_tree(branch, ' '*len(to_print))
            else:
                print(to_print + str(branch.results))

In [ ]:
print_tree(build_tree(data, headers))